# Using Scattertext to Explore the Effectiveness of Headlines
### Jason S. Kessler ([@jasonkessler](http://www.twitter.com/JasonKessler))

The code in this notebook shows how you can use the Python package Scattertext to explore how language used in headlines 
can correlate with social engagement.

For background on the term-class association scores used and semiotic squares, please see https://github.com/JasonKessler/PuPPyTalk and https://github.com/JasonKessler/SemioticSquaresTalk

This notebook makes heavy use of the library Scattertext (https://github.com/JasonKessler/scattertext) for language processing and visualizations.

The data used were scraped from Facebook by Max Woolf.  Please see his original notebook at https://github.com/minimaxir/clickbait-cluster.

In [3]:
import pandas as pd
import numpy as np
import sys
import umap
import spacy
import scattertext as st
from gensim.models import word2vec
import re
from glob import glob
from scipy.stats import rankdata
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import matplotlib.pyplot as plt

In [4]:
nlp = spacy.load('en')

In [5]:
# you may need to clone https://github.com/JasonKessler/PuPPyTalk to find this data
df = pd.concat([pd.read_csv(fn, sep='\t')
                .assign(publication=fn.split('/')[-1].split('_')[0]) 
                for fn in glob('../../PuPPyTalk/notebooks/fb_headlines/*')]).reset_index()
df['status_published'] = pd.to_datetime(df.status_published)

In [6]:
df_2016 = df[df.status_published.apply(lambda x: x.year >= 2016)].drop_duplicates()
df_2016 = df_2016.loc[df_2016['link_name'].dropna().index]
df_2016.publication.value_counts()

NYTimes     10326
CNN          9284
BuzzFeed     5386
Upworthy      824
Name: publication, dtype: int64

In [7]:
df_2016['parse'] = df_2016['link_name'].apply(nlp)

In [8]:
# Restrict to headlines longer than two words
df_2016 = df_2016[df_2016['parse'].apply(len) > 2]

In [9]:
from scipy.stats import rankdata
df_2016['reaction_percentile'] = df_2016.groupby('publication')['num_reactions'].apply(lambda x: pd.Series(rankdata(x)/len(x), index=x.index))
df_2016['reaction_bin'] = df_2016.reaction_percentile.apply(lambda x: 'Hi' if x > 2./3 else 'Lo' if x < 1./3 else 'Mid')

In [10]:
df_2016 = df_2016[df_2016.page_id.isin(['BuzzFeed', 'NYTimes'])]

In [25]:
reaction_corpus = (st.CorpusFromParsedDocuments(df_2016, 
                                                parsed_col='parse', 
                                                category_col='reaction_bin',
                                                feats_from_spacy_doc=st.PhraseMachinePhrases()
                                               )
                   .build()
                   .compact(st.ClassPercentageCompactor(term_count=2))
                   .compact(st.CompactTerms(slack=5)))

In [26]:
len(reaction_corpus.get_terms())

634

In [27]:
def get_metadata_from_corpus(corpus):
    df = corpus.get_df()
    return (df.page_id + ', ' 
            + df.reaction_percentile.apply(lambda x: str(int(x * 100)) + '%') + ', ' 
            + df.status_published.apply(lambda x: str(x.date())))

In [28]:
html = st.produce_frequency_explorer(reaction_corpus,
                                     category='Hi',
                                     not_categories=['Lo'],
                                     neutral_categories=['Mid'],
                                     neutral_category_name='Mid',
                                     minimum_term_frequency=0,
                                     pmi_filter_thresold=0,
                                     use_full_doc = True,
                                     term_scorer = st.ScaledFScorePresets(beta=1, one_to_neg_one=True),
                                     grey_threshold=0,
                                     width_in_pixels=1000,
                                     metadata=get_metadata_from_corpus(reaction_corpus),
                                     show_neutral=True,
                                     show_characteristic=False)
file_name = 'reaction_scaled_f_score.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [33]:
reaction_corpus_unigram = (st.CorpusFromParsedDocuments(df_2016, parsed_col='parse', category_col='reaction_bin')
                           .build()
                           .compact(st.ClassPercentageCompactor(term_count=3))).get_unigram_corpus()

In [35]:
html = st.produce_frequency_explorer(reaction_corpus_unigram,
                                     category='Hi',
                                     not_categories=['Lo'],
                                     neutral_categories=['Mid'],
                                     neutral_category_name='Mid',
                                     minimum_term_frequency=6,
                                     pmi_filter_thresold=0,
                                     use_full_doc = True,
                                     term_scorer = st.ScaledFScorePresets(beta=1, one_to_neg_one=True),
                                     grey_threshold=0,
                                     width_in_pixels=1000,
                                     metadata=get_metadata_from_corpus(reaction_corpus),
                                     show_neutral=True,
                                     show_characteristic=False)
file_name = 'reaction_scaled_f_score_unigram.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [42]:
html = st.produce_projection_explorer(reaction_corpus_unigram.compact(st.ClassPercentageCompactor(term_count=4)),
                                      category='Hi', 
                                      not_categories=['Lo'], 
                                      neutral_categories=['Mid'],
                                      term_scorer = st.RankDifference(),
                                      neutral_category_name='Mid',
                                      width_in_pixels=1000,
                                      use_full_doc=True,
                                      projection_model = umap.UMAP(metric='cosine'),
                                      term_acceptance_re=re.compile(''),
                                      metadata=get_metadata_from_corpus(reaction_corpus))
file_name = 'reaction_umap_projection.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [54]:
# Eliminate other categories from dataset (e.g., Upworthy or mid-engagment)
df_2016['category'] = df_2016.publication + ' ' + df_2016.reaction_bin
df_2016_four_square = df_2016[df_2016.publication.isin(['BuzzFeed', 'NYTimes']) 
                              & df_2016.reaction_bin.isin(['Hi', 'Lo'])]
# Create corpus and filter terms
four_square_corpus = (st.CorpusFromParsedDocuments(df_2016_four_square, category_col = 'category', parsed_col = 'parse')
                      .build()
                      .compact(st.CompactTerms(minimum_term_count=2, slack=6))
                      .compact(st.ClassPercentageCompactor(term_count=2)))

In [55]:
four_square_axes = st.FourSquareAxes(four_square_corpus, 
                                     ['NYTimes Hi'], 
                                     ['NYTimes Lo'], 
                                     ['BuzzFeed Hi'], 
                                     ['BuzzFeed Lo'], 
                                     labels = {'a': 'Appeals to all',
                                               'a_and_not_b': 'NY Times: ↑ Engagement',
                                               'b_and_not_a': 'NY Times: ↓ Engagement',
                                               'a_and_b': 'BuzzFeed: ↑ Engagement',
                                               'not_a_and_not_b': 'BuzzFeed: ↓ Engagement',
                                               'not_a': 'Ignored by all',
                                               'b': 'Ignored by elite, appeals to masses',
                                               'not_b': 'Appeals to elite, ignored by masses'})
html = st.produce_four_square_axes_explorer(
    four_square_axes=four_square_axes,
    x_label='NYT: Hi-Lo',
    y_label='Buzz: Hi-Lo',
    use_full_doc=True,
    pmi_threshold_coefficient=0,
    metadata=get_metadata_from_corpus(four_square_corpus),
    censor_points=False)

In [56]:
file_name = 'reaction_axes.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1600, height=900)

In [59]:
# View chart with multiple terms visible
# Set up chart structure
four_square = st.FourSquare(
    four_square_corpus,
    category_a_list=['NYTimes Hi'],
    category_b_list=['BuzzFeed Hi'],
    not_category_a_list=['BuzzFeed Lo'],
    not_category_b_list=['NYTimes Lo'],
    scorer=st.RankDifference(),
    labels={'a': 'Highbrow Engagment',
            'b': 'Lowbrow Engagment',
            'not_a_and_not_b': 'Few Facebook Reactions',
            'a_and_b': 'Many Facebook Reactions',
            'a_and_not_b': 'NYTimes',
            'b_and_not_a': 'BuzzFeed',
            'not_a': 'Lowbrow Ignored',
            'not_b': 'Highbrow Ignored'})
html = st.produce_four_square_explorer(four_square=four_square,
                                       x_label='NYTimes-Buzz',
                                       y_label='Hi-Low',
                                       use_full_doc=True,
                                       pmi_threshold_coefficient=0,
                                       metadata=get_metadata_from_corpus(four_square_corpus), 
                                       censor_points=False)
file_name = 'reaction_semiotic_censor.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1600, height=900)